<a href="https://colab.research.google.com/github/JokeBear777/Deep-Learning-Experiment/blob/main/Voice_Conversion_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hpjang/VAE_experiment
%cd CycleVAE

Cloning into 'VAE_experiment'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 231 (delta 64), reused 0 (delta 0), pack-reused 120
Receiving objects: 100% (231/231), 93.35 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (111/111), done.
[Errno 2] No such file or directory: 'CycleVAE'
/content


In [2]:
!pip install torch torchvision numpy scipy matplotlib scikit-learn librosa

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
!wget https://datashare.ed.ac.uk/bitstream/handle/10283/3061/vcc2018_database_training.zip -O vcc2018_database_training.zip
!mkdir -p corpus/train
!unzip vcc2018_database_training.zip -d corpus/train

#wget https://datashare.ed.ac.uk/bitstream/handle/10283/3061/vcc2018_database_training.zip -O training.zip
#unzip vcc2018_database_training.zip -d corpus

--2024-05-29 15:04:54--  https://datashare.ed.ac.uk/bitstream/handle/10283/3061/vcc2018_database_training.zip
Resolving datashare.ed.ac.uk (datashare.ed.ac.uk)... 129.215.67.172
Connecting to datashare.ed.ac.uk (datashare.ed.ac.uk)|129.215.67.172|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 122768173 (117M) [application/zip]
Saving to: ‘vcc2018_database_training.zip’

vcc2018_database_tr 100%[===================>] 117.08M  27.8MB/s    in 8.8s    

2024-05-29 15:05:04 (13.3 MB/s) - ‘vcc2018_database_training.zip’ saved [122768173/122768173]

Archive:  vcc2018_database_training.zip
   creating: corpus/train/vcc2018_training/
   creating: corpus/train/vcc2018_training/VCC2SF3/
  inflating: corpus/train/vcc2018_training/VCC2SF3/20068.wav  
  inflating: corpus/train/vcc2018_training/VCC2SF3/20031.wav  
  inflating: corpus/train/vcc2018_training/VCC2SF3/20039.wav  
  inflating: corpus/train/vcc2018_training/VCC2SF3/20021.wav  
  inflating: corpus/train/vcc2018_t

In [4]:
!wget https://datashare.ed.ac.uk/bitstream/handle/10283/3061/vcc2018_database_evaluation.zip -O vcc2018_database_evaluation.zip
!mkdir -p corpus/test
!unzip vcc2018_database_evaluation.zip -d corpus/test

#wget https://datashare.ed.ac.uk/bitstream/handle/10283/3061/vcc2018_database_evaluation.zip -O test.zip
#unzip vcc2018_database_evaluation.zip -d corpus

--2024-05-29 15:05:11--  https://datashare.ed.ac.uk/bitstream/handle/10283/3061/vcc2018_database_evaluation.zip
Resolving datashare.ed.ac.uk (datashare.ed.ac.uk)... 129.215.67.172
Connecting to datashare.ed.ac.uk (datashare.ed.ac.uk)|129.215.67.172|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 33338426 (32M) [application/zip]
Saving to: ‘vcc2018_database_evaluation.zip’

vcc2018_database_ev 100%[===================>]  31.79M  5.70MB/s    in 6.7s    

2024-05-29 15:05:19 (4.76 MB/s) - ‘vcc2018_database_evaluation.zip’ saved [33338426/33338426]

Archive:  vcc2018_database_evaluation.zip
   creating: corpus/test/vcc2018_evaluation/
   creating: corpus/test/vcc2018_evaluation/VCC2SF3/
  inflating: corpus/test/vcc2018_evaluation/VCC2SF3/30023.wav  
  inflating: corpus/test/vcc2018_evaluation/VCC2SF3/30011.wav  
  inflating: corpus/test/vcc2018_evaluation/VCC2SF3/30005.wav  
  inflating: corpus/test/vcc2018_evaluation/VCC2SF3/30008.wav  
  inflating: corpus/test/

In [5]:
!pip install -U -q PyDrive2

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.0 MB/s eta 0:00:00


In [7]:
!unzip /content/gdrive/MyDrive/testSet-20240527T061049Z-001.zip -d corpus/korean



Archive:  /content/gdrive/MyDrive/testSet-20240527T061049Z-001.zip
  inflating: corpus/korean/testSet/M2/mv02_t10_s66.wav  
  inflating: corpus/korean/testSet/M2/mv02_t10_s30.wav  
  inflating: corpus/korean/testSet/M2/mv02_t10_s23.wav  
  inflating: corpus/korean/testSet/M2/mv02_t11_s14.wav  
  inflating: corpus/korean/testSet/F2/fy03_t10_s23.wav  
  inflating: corpus/korean/testSet/F2/fy03_t10_s03.wav  
  inflating: corpus/korean/testSet/F2/fy03_t09_s08.wav  
  inflating: corpus/korean/testSet/F2/fy03_t09_s50.wav  
  inflating: corpus/korean/testSet/M2/mv02_t09_s08.wav  
  inflating: corpus/korean/testSet/M2/mv02_t10_s12.wav  
  inflating: corpus/korean/testSet/F2/fy03_t09_s34.wav  
  inflating: corpus/korean/testSet/F2/fy03_t10_s30.wav  
  inflating: corpus/korean/testSet/M2/mv02_t09_s09.wav  
  inflating: corpus/korean/testSet/F2/fy03_t10_s27.wav  
  inflating: corpus/korean/testSet/M2/mv02_t09_s34.wav  
  inflating: corpus/korean/testSet/M2/mv02_t11_s20.wav  
  inflating: corpus/k

In [8]:
!unzip /content/gdrive/MyDrive/trainingSet-20240526T074732Z-001.zip -d corpus/korean

Archive:  /content/gdrive/MyDrive/trainingSet-20240526T074732Z-001.zip
  inflating: corpus/korean/trainingSet/F2/fy03_t12_s02.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t16_s16.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t18_s06.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t11_s24.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t13_s11.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t12_s07.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t16_s32.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t10_s07.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t15_s03.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t12_s12.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t13_s18.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t14_s20.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t09_s23.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t18_s13.wav  
  inflating: corpus/korean/trainingSet/F2/fy03_t18_s05.wav  
  inflating: c

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [9]:
# 필요한 패키지 설치
!pip install pyworld
!pip install pyworld pysptk
!pip install pyworld pysptk colorama --upgrade




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyworld: filename=pyworld-0.3.4-cp310-cp310-linux_x86_64.whl size=865337 sha256=484b87100d1b6a8baadfa3da0abadefadf07f7934a8660fa84cfd286fd1615bb
  Stored in directory: /root/.cache/pip/wheels/66/09/8a/a1d79b73d59756f66e9bfe55a199840efc7473adb76ddacdfd
Successfully built pyworld
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.3/421.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysptk: filename=pysptk-0.2.2-cp310-cp310-linux_x86_64.whl size=1235506 sha256=238a2fc94b877f94783b986a85c016ce868182547af5897a397aae1e9a89b2e0
  Stored in directory: /root/.cache/pip/wheels/87/ef/11/e708873d0361690e25e06ccfd1d793ff4549a91bb48ee58

In [17]:
# 데이터 전처리 스크립트 실행
!python3  /content/VAE_experiment/preprocess/preprocess-vcc2018.py


Loading VCC2SF1 Wavs...
Processing 10064
Processing 10020
Processing 10066
Processing 10074
Processing 10080
Processing 10073
Processing 10023
Processing 10054
Processing 10058
Processing 10052
Processing 10032
Processing 10004
Processing 10027
Processing 10021
Processing 10065
Processing 10031
Processing 10048
Processing 10026
Processing 10078
Processing 10049
Processing 10040
Processing 10075
Processing 10009
Processing 10030
Processing 10006
Processing 10043
Processing 10001
Processing 10007
Processing 10005
Processing 10018
Processing 10063
Processing 10046
Processing 10053
Processing 10072
Processing 10003
Processing 10051
Processing 10033
Processing 10044
Processing 10016
Processing 10041
Processing 10071
Processing 10060
Processing 10008
Processing 10036
Processing 10038
Processing 10056
Processing 10029
Processing 10019
Processing 10045
Processing 10068
Processing 10061
Processing 10070
Processing 10028
Processing 10014
Processing 10002
Processing 10057
Processing 10037
Process

In [18]:
log_dir = '/content/VAE_experiment/log'
!mkdir -p {log_dir}

# run_all.sh 스크립트를 백그라운드에서 실행하고 로그를 vae_exp_0520.txt 파일에 저장
!nohup bash /content/VAE_experiment/run_all.sh > {log_dir}/vae_exp_0520.txt &

nohup: redirecting stderr to stdout


In [15]:
!python3 -u /content/convert.py --model_type=VAE3 --model_path=/content/model/VAE3

Processing VCC2SF1
	Convert 30029.wav ...
/content/convert.py:116: RuntimeWarning: divide by zero encountered in log
  logf0_norm = (np.log(f0)-logf0_m_s) / logf0_s_s
/content/convert.py:120: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  y = torch.Tensor(style).float().to(device).contiguous()
	Convert 30025.wav ...
	Convert 30033.wav ...
	Convert 30013.wav ...
	Convert 30028.wav ...
	Convert 30020.wav ...
	Convert 30010.wav ...
	Convert 30017.wav ...
	Convert 30009.wav ...
	Convert 30021.wav ...
	Convert 30030.wav ...
	Convert 30027.wav ...
	Convert 30034.wav ...
	Convert 30023.wav ...
	Convert 30008.wav ...
	Convert 30014.wav ...
	Convert 30019.wav ...
	Convert 30031.wav ...
	Convert 30026.wav ...
	Convert 30004.wav ...
	Convert 30024.wav ...
	Convert 30011.wav ...
	Convert 

In [13]:
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512612 sha256=07e7eb02b201a8aea80daf74753b1fd18fe9c2af4222c577e3c892ebf692d59b
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [27]:
!python3 -u /content/calculate/calculate.py --test_dir=/content/result/VAE3 > /content/stats/VAE3_0.txt


In [30]:
!bash /content/run_graph.sh


In [58]:
!python3 -u /content/train_further.py --model_type=VAE3 --conf=/content/VAE_experiment/conf/CC.json --seed=0 --model_dir=/content/model/VAE3

Namespace(model_type='VAE3', SI=0, I=0, LI=0, AC=0, SC=0, CC=1, GAN=0, seed=0, conf='/content/VAE_experiment/conf/CC.json', model_dir='/content/model/VAE3', lr=0)
/content/model/VAE3
EPOCH: 0
/content/data_manager.py:200: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  A_y = torch.Tensor(A_y).float().to(device)
Train: rec_loss : 1.602 / kl_loss : 0.3074 / CC_loss : 4.8062 / total_loss : 6.4389 /  
DEV: rec_loss : 1.4837 / kl_loss : 0.2026 / CC_loss : 4.4528 / total_loss : 5.903 /  
.....................
EPOCH: 1
Train: rec_loss : 1.4628 / kl_loss : 0.1469 / CC_loss : 4.3913 / total_loss : 5.8688 /  
DEV: rec_loss : 1.461 / kl_loss : 0.0917 / CC_loss : 4.387 / total_loss : 5.721 /  
.....................
EPOCH: 2
Train: rec_loss : 1.4507 / kl_loss : 0.0662 / CC_loss : 4.3579 / t